In [ ]:
import pandas as pd
import requests
from tqdm import tqdm
tqdm.pandas()
import json

In [ ]:
# url with a placeholder for cursor
example_url_with_cursor = 'https://api.openalex.org/works?filter=abstract.search:renewable%20energy&per-page=200&cursor={}&mailto=kuntakim88@gmail.com'
cursor = '*'

In [ ]:
 # set cursor value and request page from OpenAlex
url = example_url_with_cursor.format(cursor)
print('\n' + url)
page_with_results = requests.get(url).json()
page_with_results

In [ ]:
!mkdir renewable_json

In [ ]:
# json으로 저장
with open('./renewable_json/renewable_1.json','w') as f:
    json.dump(page_with_results['results'],f)

In [ ]:
# end = int(239325/200)+1 # 일일 10만개, 1회 200개 요청
end = 5
start = 1
print(start, end)

In [ ]:
for idx in tqdm(range(start,end)):
     # set cursor value and request page from OpenAlex
    url = example_url_with_cursor.format(cursor)
    page_with_results = requests.get(url).json()
    
    # json으로 저장
    with open(f'./renewable_json/renewable_{idx}.json','w') as f:
        json.dump(page_with_results,f)
    # next page
    cursor = page_with_results['meta']['next_cursor']

In [ ]:
# json 파일 리스트
file_list = glob.glob('./renewable_json/*.json')
len(file_list)
file_list

In [ ]:
# 여러 json 파일을 읽어서 result만 하나의 리스트로 저장하기
openalex_json = []
for file in tqdm(file_list):
    with open(file,'r') as f:
        temp_json = json.load(f)['results']
    openalex_json.extend(temp_json)

In [ ]:
openalex_json

In [ ]:
# pandas dataframe으로 변환
df = pd.DataFrame.from_dict(openalex_json)
df

In [ ]:
# 중복삭제 (id 컬럼 기준)
df = df.drop_duplicates(subset=['id']).reset_index(drop=True)
df

In [ ]:
# 초록 텍스트 복원(리스트형태에서 텍스트로)
def index_to_text(abstract_inverted_index):
    abstract_index = {}
    for k, vlist in abstract_inverted_index.items():
        for v in vlist:
            abstract_index[v] = k
    abstract = ' '.join(abstract_index[k] for k in sorted(abstract_index.keys()))
    return abstract

In [ ]:
# 초록 복원 테스트
index_to_text(df['abstract_inverted_index'][0])

In [ ]:
# 초록 복원 결과를 'abstract' 컬럼에 추가
df['abstract'] = df[~pd.isna(df['abstract_inverted_index'])]['abstract_inverted_index'].progress_map(lambda x:index_to_text(x))

In [ ]:
df['abstract']

In [ ]:
df